<a href="https://colab.research.google.com/github/vincent-arese/OC-P04VA/blob/main/P4_VA_FAOdata-archivesPyplot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P4-VA Panorama de l’état de la malnutrition dans le monde (FAO 2017)

Dataset : FAO stat : 

# Environnement de travail et  Data set

### pip install

In [ ]:
#Prepa Env PIP
# librairie pour combiner Pyhton & SQL https://towardsdatascience.com/fugue-and-duckdb-fast-sql-code-in-python-e2e2dfc0f8eb
#!pip install -U fugue[duckdb,sql] 
# !pip install pandas-bokeh

#  TO DEL    !pip install -U dash 

#! pip list -v
# ! pip list -v | grep [Pp]an 
# ! pip list -v | grep [Pp]lot
# ! pip list -v | grep [Ss]ea

# ! pip install --upgrade pandas

### Librairies

In [9]:
#Charger les librairies Pythons 
import numpy as np
import pandas as pd
from pandas.core.groupby.generic import DataFrameGroupBy
pd.set_option('display.max_columns', None)  #pd.set_option('max_columns', 10) limiter le nbr de colonnes visualisé à 10 /none (=> scrollbar)
pd.set_option('display.max_rows', 40)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas_bokeh
#import fugue_duckdb
#from fugue_notebook import setup ;setup()

### Fonction Personalisées

In [10]:
#Fonction : Décrire un Data Frame
def DECRIRE(DataFrame):
  """"Description personalisé d'un DataFrame 
  DECRIRE(DataFrame)  : Nom du dataframe
  frait un print  :  DataFrame.info()  - DataFrame.Describeo()  - Sommpe des Nulls - Data.colomuns 
  """
  data=DataFrame
  return print(data.info(),'\n \n Describe :\n', data.describe(), '\n\n Nb de NaN:',data.isnull().sum(axis=0),'\n\n liste_des_Colonnes :', data.columns)  # Compter les nan   

In [11]:
# Fonction Recherche Valeur dans un DataFrame
def RECHERCHEV(DataFrame,Valeur):
 dfx=DataFrame.mask(Valeur)
 return dfx

In [12]:
# #Recherche valeur (val) dans un data frame
# val=0 ;df=dispo
# print(df[df['Habitant_millions'] == val].sum())
# print(df[df["Exports"] == val].count()) ; #df[df["Exports"] == val]

## Data Set FAO

In [13]:
#  Importer les données en Pandas dataframe
#aideDataPATH='/content/drive/MyDrive/DataSc/P4VA/'
DataPATH='https://raw.githubusercontent.com/vincent-arese/OC-P04VA/main/FAOdata/'

#DEBUG print(DataPATH+'SecuAlimUTF8.csv',': Pd.read OK')

SecuAlimRAW=pd.read_csv(DataPATH+'SecuAlimUTF8.csv',sep=';')
# BilanAlimRAW=pd.read_csv(DataPATH+'BilansAlimentaires-UTF8.zip',sep=',') #####  ERUEUR
AideAlimRAW=pd.read_csv(DataPATH+'FAOSTAT_aideAlimentaire.csv',sep=',',usecols = [i for i in range(2,12)])


#Population mondiale



ListeProduitsFAO=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/FAOProduits.xlsx?raw=true', sheet_name='listeProduitMini',usecols = [i for i in range(5)]) # Codes produits FAO
PAYS=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/PAYS%20_FAO.xlsx?raw=true', sheet_name='PaysFAO')   # DataSet pour plot map monde 

print("https://github.com/vincent-arese/OC-P04VA/tree/main/FAOdata") # Lien vers le repertoire Github



https://github.com/vincent-arese/OC-P04VA/tree/main/FAOdata


### Population Mondiale

In [14]:
# Population Mondiale RAW dataSet FAO  source : https://www.fao.org/faostat/fr/#data/FBSH
popI=pd.read_csv(DataPATH+'FAOSTAT_pop1961-2013.csv',sep=',')
popII=pd.read_csv(DataPATH+'FAOSTAT_pop2010-2019.csv',sep=',')
popII.rename(columns={'Code zone (FAO)':'Code zone'},inplace=True) # Renommer la colonne qui servira de fK pour la jonction 

#Debug
# popI.columns==popII.columns  # Ok
# popI[(popI.Année<2010)].Année.unique()   # [1961,2009]
# popII.Année.unique()  # [2010,2019]
#pop=pd.merge(popI, popII, how='outer')

# Merge des deux CSV
pop=pd.merge(popI[popI.Année<=2009], popII, how='outer')
# pop[(pop.Zone=='Afghanistan') & (pop.Année>=2005)] # Verification pas de doublon


# Nettoyage Data Frame pop

pop=pop.iloc[:,[i for i in range(2,12)]] # supression rapide des colonnes inutiles

# pop.columns
"""Index(['Code Domaine', 'Domaine', 'Code zone', 'Zone', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole', 'Code zone (FAO)'],
      dtype='object')"""

pop.drop(columns=['Code année','Code Élément','Élément','Unité', 'Code Produit', 'Produit'],inplace=True) # supression colonnes inutiles restantes

pop.rename(columns={'Code zone':'CodeZone','Valeur':'Population'},inplace=True) # Reommer colonnes
pop.Population=(pop.Population*1000) # Convertion Population  (  car était exprimé en milliers )
pop['PopulationM']=pop.Population/10**6  # Creation Pop Million
pop


# Recherche des NaN
# pop.isnull().sum(axis=0) # recherche & sommme des Nan ds chaque colonne   # OK pas de NaN
# pop[pop.isnull().any(axis=1)] # Visuliser les lignes contenant au moins un NaN

# DEBUG  recherche origine CodeZone Manquant suite à la jointure  
# print(pop[pop.isnull().any(axis=1)].Zone.unique(),'Nombre :',len(pop[pop.isnull().any(axis=1)].Zone.unique())) # Visuliser les lignes contenant au moins un NaN  / listes des pays concernés 

,CodeZone,Zone,Année,Population,PopulationM
0,2,Afghanistan,1961,8954000.0,8.954
1,2,Afghanistan,1962,9142000.0,9.142
2,2,Afghanistan,1963,9340000.0,9.340
3,2,Afghanistan,1964,9547000.0,9.547
4,2,Afghanistan,1965,9765000.0,9.765
...,...,...,...,...,...
9710,181,Zimbabwe,2015,13815000.0,13.815
9711,181,Zimbabwe,2016,14030000.0,14.030
9712,181,Zimbabwe,2017,14237000.0,14.237
9713,181,Zimbabwe,2018,14439000.0,14.439


### PIB par habitant, (US$ PPA internationaux constants de 2015)

In [15]:
# PIB par habitant, (US$ PPA internationaux constants de 2015) dataSet FAO  source : https://www.fao.org/faostat/fr/#data/PD
pib=pd.read_csv(DataPATH+'FAOSTAT_PIBhab-1970-2019.csv',sep=',',usecols = [i for i in range(2,12)]) 


# Nettoyage PIB 
# pibRAW.columns """Index(['Code zone (FAO)', 'Zone', 'Code ?l?ment', '?l?ment', 'Code Produit','Produit', 'Code ann?e', 'Ann?e', 'Unit?', 'Valeur'], dtype='object')"""
pib.drop(columns=['Code ?l?ment', '?l?ment', 'Code ann?e', 'Unit?', 'Code Produit','Produit'],inplace=True) # supression colonnes inutiles restantes
pib.rename(columns={'Code zone (FAO)':'CodeZone','Ann?e':'Année','Valeur':'USDparHab'},inplace=True) # Renommer la colonne qui servira de fK pour la jonction 
pib

,CodeZone,Zone,Année,USDparHab
0,2,Afghanistan,1970,16.5409
1,2,Afghanistan,1971,17.3665
2,2,Afghanistan,1972,18.1058
3,2,Afghanistan,1973,19.0936
4,2,Afghanistan,1974,22.0401
...,...,...,...,...
10009,181,Zimbabwe,2015,100.0000
10010,181,Zimbabwe,2016,102.1610
10011,181,Zimbabwe,2017,104.6570
10012,181,Zimbabwe,2018,110.1200


#### test des NaN  

In [16]:
# DEBUG  recherche NaN   - OK 
# temp=pib
# temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# # test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

### Liste Pays

In [17]:
# Liste des Pays avec CodeZone FAO  & AlphaISO3 pour plot sur map Monde 
# source : https://www.fao.org/faostat/fr/#data/FS  Rubrique : Définitions et Standards  Formaté sous Excel
PAYS=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/PAYS%20_FAO.xlsx?raw=true', sheet_name='PaysFAO')   # DataSet pour plot map monde 

PAYS.head(1)

,CodeContinent,continent,Code_ZoneContinent,ZoneContinent,Code_Pays,Pays,AlphaISO3
0,5100,Afrique,5104,Afrique australe,202,Afrique du Sud,ZAF


### Recap Géopolitique

In [18]:
# Recap Data Géopolitique : Population, PIB par Hab (USD 20015), Liste PAYS

# PAYS.columns 
""" Index(['CodeContinent', 'continent', 'Code_ZoneContinent', 'ZoneContinent','Code_Pays', 'Pays', 'AlphaISO3'], dtype='object'"""
# pop.columns  
"""Index(['CodeZone', 'Zone', 'Année', 'Population', 'PopulationM'], dtype='object')"""

#pib.columns  
"""Index(['CodeZone', 'Zone', 'Année', 'USDparHab'], dtype='object')"""

# Join :  (PAYS & pop) & pib
geo=PAYS.merge(pop,left_on='Code_Pays',right_on='CodeZone').merge(pib,left_on='CodeZone',right_on='CodeZone')
geo.drop(columns=['Code_Pays','Pays','Année_y','Zone_y'],inplace=True)
geo.rename(columns={'Zone_x':'Zone','Année_x':'Année','continent':'Continent'},inplace=True)
geo.Population=geo.Population.astype(int)
geo


# geo.info()

# DEBUG  recherche NaN
# temp=geo
# temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

,CodeContinent,Continent,Code_ZoneContinent,ZoneContinent,AlphaISO3,CodeZone,Zone,Année,Population,PopulationM,USDparHab
0,5100,Afrique,5104,Afrique australe,ZAF,202,Afrique du Sud,1961,17850000,17.850,17.8579
1,5100,Afrique,5104,Afrique australe,ZAF,202,Afrique du Sud,1961,17850000,17.850,18.9013
2,5100,Afrique,5104,Afrique australe,ZAF,202,Afrique du Sud,1961,17850000,17.850,19.5235
3,5100,Afrique,5104,Afrique australe,ZAF,202,Afrique du Sud,1961,17850000,17.850,25.5901
4,5100,Afrique,5104,Afrique australe,ZAF,202,Afrique du Sud,1961,17850000,17.850,30.2913
...,...,...,...,...,...,...,...,...,...,...,...
450300,5500,Océanie,5504,Polynésie,WSM,244,Samoa,2019,197000,0.197,100.0000
450301,5500,Océanie,5504,Polynésie,WSM,244,Samoa,2019,197000,0.197,100.7050
450302,5500,Océanie,5504,Polynésie,WSM,244,Samoa,2019,197000,0.197,101.6940
450303,5500,Océanie,5504,Polynésie,WSM,244,Samoa,2019,197000,0.197,102.1420


#### test des NaN  

In [ ]:
# DEBUG  recherche NaN ==> Ok 
# temp=geo
# temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

#### test Liste produits TO DEL

In [ ]:
# GproduitRAW=pd.read_csv(DataPATH+'FAOSTAT_listeGroupeProduits.csv',sep=',',usecols = [i for i in range(5)])
# # GproduitRAW
# Produits=pd.read_csv(DataPATH+'FAOSTAT_ListeProduits.csv',sep=',',usecols = [i for i in range(3)])
# # Produits

# ListeProduits=pd.merge(GproduitRAW,Produits, how='left',on='Code Produit')
# ListeProduits

In [ ]:
# ListeProduits.info()
# ListeProduits.isnull().sum(axis=0)

In [ ]:
# ListeProduits[ListeProduits['Produit_y'].isnull()] 

In [ ]:
# ListeProduits['Groupe de produits'].unique() 

In [ ]:
# ListeProduits[(ListeProduits['Code Produit']==1743)] 

In [ ]:
# encoding='ANSI'

# pd.read_csv(DataPATH+'BilansAlimentaires_F_Toutes_les_Données_(Normalisé).zip',sep=',',encoding='utf-8')

#Doc python Encoding :  https://docs.python.org/3/library/codecs.html#standard-encodings

### Aide Alimentaire

In [19]:
# Aide Alimentaire (WFP) Expédition RAW dataSet FAO  source : https://www.fao.org/faostat/fr/#data/FA
AideAlimRAW=pd.read_csv(DataPATH+'FAOSTAT_aideAlimentaire.csv',sep=',',usecols = [i for i in range(2,12)])

# AideAlimRAW
# AideAlimRAW.info()
#AideAlimRAW.columns  #Index(['Code Pays bénéficiaire (FAO)', 'Pays bénéficiaire', 'Code Élément','Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité', 'Valeur'], dtype='object')

AideAlimRAW.drop(columns=['Code année','Code Élément','Élément','Unité'],inplace=True) #Suppresion colonnes inutiles
AideAlimRAW.rename(columns={'Code Pays bénéficiaire (FAO)' : 'CodePays', 'Pays bénéficiaire': 'PaysBénéficiaire', 'Code Produit':'CodeProduitAA', 'Valeur':'tonnes'},inplace=True)
AideAlim=AideAlimRAW

AideAlim
# Attention CodeProduitAA  pas compatible avec CodeProduit (utilisé dans les autres fichiers de la FAO))

,CodePays,PaysBénéficiaire,CodeProduitAA,Produit,Année,tonnes
0,2,Afghanistan,11704,Autres non-céréales,2000,55
1,2,Afghanistan,11704,Autres non-céréales,2002,2313
2,2,Afghanistan,11704,Autres non-céréales,2003,9821
3,2,Afghanistan,11704,Autres non-céréales,2004,7332
4,2,Afghanistan,11704,Autres non-céréales,2005,16985
...,...,...,...,...,...,...
10418,181,Zimbabwe,11890,"Sucre, total",2002,136
10419,181,Zimbabwe,11890,"Sucre, total",2003,216
10420,181,Zimbabwe,11890,"Sucre, total",2008,5
10421,181,Zimbabwe,11890,"Sucre, total",2009,19


In [ ]:
# DEBUG  recherche NaN  => ok
# temp=AideAlim
# temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

In [ ]:
# Jointure pour data visualisation sur map 
# GeoTable=tableZ.merge(PAYS, left_on='CodeZone',right_on='Code_Pays')
# GeoTable.drop(['CodeContinent','Code_ZoneContinent','Code_Pays','Pays'], axis=1)

AideAlimGeo=AideAlim.merge(PAYS, left_on='CodePays',right_on='Code_Pays').drop(['CodeContinent','Code_ZoneContinent','Code_Pays','Pays'], axis=1) #.fillna(0)
AideAlimGeo



,CodePays,PaysBénéficiaire,CodeProduitAA,Produit,Année,tonnes,continent,ZoneContinent,AlphaISO3
0,2,Afghanistan,11704,Autres non-céréales,2000,55,Asie,Asie méridionale,AFG
1,2,Afghanistan,11704,Autres non-céréales,2002,2313,Asie,Asie méridionale,AFG
2,2,Afghanistan,11704,Autres non-céréales,2003,9821,Asie,Asie méridionale,AFG
3,2,Afghanistan,11704,Autres non-céréales,2004,7332,Asie,Asie méridionale,AFG
4,2,Afghanistan,11704,Autres non-céréales,2005,16985,Asie,Asie méridionale,AFG
...,...,...,...,...,...,...,...,...,...
9789,181,Zimbabwe,11890,"Sucre, total",2002,136,Afrique,Afrique orientale,ZWE
9790,181,Zimbabwe,11890,"Sucre, total",2003,216,Afrique,Afrique orientale,ZWE
9791,181,Zimbabwe,11890,"Sucre, total",2008,5,Afrique,Afrique orientale,ZWE
9792,181,Zimbabwe,11890,"Sucre, total",2009,19,Afrique,Afrique orientale,ZWE


 #### test NaN

In [20]:
#DEBUG recherche des nan  => ok 
# AideAlimGeo.isnull().sum(axis=0)
# AideAlimGeo.isnull().sum(axis=1).unique()
# AideAlimGeo.Année.unique()

### Bilan Alimentaire 

In [21]:
# bilan=pd.read_csv('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/FAOBilanAlimentaireBILAN.zip?raw=true',sep=',')
# dispo=pd.read_csv('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/FAOBilanAlimentaireDISPO.zip?raw=true',sep=',')

# code errreur !!! 

# FAOBilanAlimentaireDISPO.zip

In [1]:
# Les fichiers ont été normalisé et reduit aux clef  pour optimiser le chargement 

import numpy as np
import pandas as pd
DataPATH='https://raw.githubusercontent.com/vincent-arese/OC-P04VA/main/FAOdata/'


# Chargment des fichiers 
bilan=pd.read_csv(DataPATH+'FAOBilanAlimentaireBilanUTF8.zip',sep=';',usecols = [i for i in range(5)])
dispo=pd.read_csv(DataPATH+'FAOBilanAlimentaireDispoUTF8.zip',sep=';',usecols = [i for i in range(5)])
CodeElement=pd.read_csv(DataPATH+'FAOBilanAlimentaireCodeUTF8.csv',sep=';')




In [2]:
dispo.info()
len(dispo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859628 entries, 0 to 859627
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   CodeZone     859628 non-null  int64  
 1   CodeProduit  859628 non-null  int64  
 2   CodeElément  859628 non-null  int64  
 3   Année        859628 non-null  int64  
 4   Valeur       859628 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 32.8 MB


859628

In [3]:
ListeProduitsFAO=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/FAOProduits.xlsx?raw=true', sheet_name='listeProduitMini',usecols = [i for i in range(5)]) # Codes produits FAO
# ListeProduitsFAO.head(1)


In [4]:
# Merge du Dataframes bilan , dispo , code  Normalisés

# bilan.columns
"""Index(['CodeZone', 'CodeProduit', 'CodeElément', 'Année', 'Valeur'], dtype='object')"""

# dispo.columns
"""Index(['CodeZone', 'CodeProduit', 'CodeElément', 'Année', 'Valeur'], dtype='object')"""

# ListeProduitsFAO.columns
"""Index(['Origine', 'CodeProduit', 'Produit', 'Coef', 'Description'], dtype='object')"""

# CodeElement.columns
"Index(['CodeElément', 'Elément', 'Unité'], dtype='object')"

# geo.columns
"""Index(['CodeContinent', 'Continent', 'Code_ZoneContinent', 'ZoneContinent',  'AlphaISO3', 'CodeZone', 'Zone', 'Année', 'Population', 'PopulationM','USDparHab'], dtype='object')"""


temp=pd.concat([bilan,dispo])



NameError: ignored

In [ ]:
temp2=temp.merge(CodeElement,how='left',on='CodeElément').merge(geo,how='left',on='CodeZone')

In [ ]:
# DEBUG  recherche NaN  
# temp=test
# temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

### Sécurité Alimentaire




#### Nettoyage & Formatage Dataframe

##### 1 Import du RAW

In [ ]:
#FAO  Securité Alimentaire   https://www.fao.org/faostat/fr/#data/FS 
# Rubrique Toutes les données 	https://fenixservices.fao.org/faostat/static/bulkdownloads/Données_de_la_sécurité_alimentaire_F_Toutes_les_Données_(Normalisé).zip
#Probleme encodage donc import dans Excel
SecuAlim=pd.read_excel('https://github.com/vincent-arese/OC-P04VA/blob/main/FAOdata/SecuAlim.xlsx?raw=true', sheet_name='SecuAlim',usecols = [i for i in range(10)])
# SecuAlim.xlsx

In [ ]:
# Nettoyage SecuAlim 

# Supprimerles lignes sans Valeur : Colonne 'Valeur' => 5159 NaN sur 148060 rows × 10 columns
SecuAlim=SecuAlim.dropna() # drop Row ayant au moins un NaN   


# Drop  les CodeZone correspondant à des agrégats ( continent, groupe de pays ........... ) =>  CodeZone>=400 ! 
''' 'Pays les moins développés' 'Pays sans littoral en voie de développement' 'Petites îles en voie de Développement'  'Pays à faible revenu et à déficit vivrier' 'Pays à faible revenu'  'Pays à revenu moyen' 'High-income economies'  'Upper-middle-income economies' '''

SecuAlim.drop(SecuAlim.loc[SecuAlim['CodeZone'] >= 400].index, inplace=True)
# print(SecuAlim.Zone.unique(),'\nTotal:',len(SecuAlim.Zone.unique())) #   Verfier Si le filtre a bien fonctionné


# Converions Période/Année en AnnéeREF 
# SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF'] : TEST pour isoler les Années simple des Periodes) 
# SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[:4].astype(int)  Isoler la Année_début de la periode
# AnnéeREF =(Année_début +Année_fin)/2 

SecuAlim.rename(columns={'CodeAnnée':'AnnéeREF'},inplace=True)
SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF']=((SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[:4].astype(int)+SecuAlim.loc[SecuAlim.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[4:8].astype(int))/2).astype(int)


SecuAlim

In [ ]:
SecuAlim['ListeProduit']=SecuAlim.CodeProduit.astype(str)+'-'+SecuAlim.Produit # Creation Colonne listeProduit pour filtrage 
SecuAlim.ListeProduit.unique()  # Print listeProduit pour filtrage 

In [ ]:
# DEBUG  Recheches des Années dispo  pour chaque CodeProduit (Non dispo = futur NaN)

df=SecuAlim
listeCodeProduit=SecuAlim.ListeProduit.unique() 
for p in listeCodeProduit:
  print(p,' : \n',df[(df.ListeProduit == p) & (df.AnnéeREF >= 2000)].AnnéeREF.unique(),'\n' )


# DEBUG Requettes Intermediaires pour faire la boucle For
# filtered_df[(filtered_df.CodeProduit == 210081) & (filtered_df.AnnéeREF >= 2000)].head(20)
# filtered_df[(filtered_df.CodeProduit == 210081) & (filtered_df.AnnéeREF >= 2000)].AnnéeREF.unique()





In [ ]:
# Restriction : Choix des CodeProduit à conserver  cf. ListeProduit au-dessus

SelectedCodeProduit=[21013,21014,210011,21056,21057,21061] ;SelectedCodeProduit

SecuAlim_selected=SecuAlim[SecuAlim.CodeProduit.isin(SelectedCodeProduit)]

#DEBUG 
# SecuAlim.CodeProduit ==21013   #etape0
# SecuAlim.CodeProduit.isin([21013,21014,210011,21056,21057,21061]) # Etape1
# SecuAlim_selected.CodeProduit.unique() #Verification restriction
SecuAlim_selected.ListeProduit.unique() #Verification restriction



In [ ]:
SecuAlim_selected.head(5)
# SecuAlim_selected.describe(include='all')  # ==> Valeur contient '<0.1' ==> objet
# SecuAlim_selected.info()   # ==>  Valeur 19193 non-null  object

In [ ]:
# DEBUG  recherche NaN  - OK 
temp=SecuAlim_selected
temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()


In [ ]:
# DEBUG  Recheches des Années dispo  pour chaque CodeProduit (Non dispo = futur NaN)



In [ ]:
SecuAlim_selected[(SecuAlim_selected.AnnéeREF==2000) & (SecuAlim_selected.CodeProduit==21061)]

In [ ]:
# Pivot table pour structurer le dataframe

# SecuAlim_selected.columns 
""" Index(['CodeZone', 'Zone', 'CodeProduit', 'Produit', 'CodeElément', 'Elément', 'AnnéeREF', 'Année', 'Unité', 'Valeur', 'ListeProduit'], dtype='object') """

SecuAlimP=SecuAlim_selected.pivot( index=['CodeZone','Zone','AnnéeREF'],columns=['ListeProduit'], values='Valeur')  
SecuAlimP



In [ ]:
# Convertion df multi-index  vers  df index simple
temp=SecuAlimP

temp.columns=temp.columns.to_flat_index() # colonnes => concatener le multi index en un index simple https://datascientyst.com/flatten-multiindex-in-pandas/
#tableZ.columns = tableZ.columns.get_level_values(0) + '_' +  tableZ.columns.get_level_values(1).astype(str)+ '_' + tableZ.columns.get_level_values(2)

temp=temp.reset_index() # lignes Conversion de multi-index en colonnes.
temp

In [ ]:
# DEBUG  recherche NaN  - OK 
# temp=SecuAlim_selected
temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
test.AnnéeREF.unique()

In [ ]:
SecuAlimP.info()

In [ ]:
# DEBUG  recherche NaN  
temp=SecuAlimP
temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
test.Zone.unique()

In [ ]:
# Chamger les Valeurs '<0.1' pour passer colonne Valeur du type objet à float
SecuAlim_selected[SecuAlim_selected.Valeur =='<0.1'] # Selectionner les rows dont Valeur == '<0.1'

SecuAlim_selected=SecuAlim_selected.replace('<0.1', '0.000001')  # Remplacer '<0.1' par  '0.000001' dans le DataFrame SecuAlim_selected

#DEBUG : Verif selection 
# SecuAlim_selected[SecuAlim_selected.Valeur =='<0.1'] # DEBUG Selectionner les rows dont Valeur == '<0.1'
# SecuAlim_selected[SecuAlim_selected.Valeur =='0.000001'] # DEBUG Selectionner les rows dont Valeur == '0.000001'

SecuAlim_selected.Valeur=SecuAlim_selected.Valeur.astype(float) # Conversion au type float


#DEBUG : Verif 
# SecuAlim_selected.info() #   Valeur 19193 non-null  float64 / len(SecuAlim_selected) #19193

SecuAlim_selected

In [ ]:
len(SecuAlim_selected)

In [ ]:
SecuAlim_selected[SecuAlim_selected.Valeur =='<0.1'] # Selection des rows 

# SecuAlim_selected[SecuAlim_selected.Valeur =='<0.1'].replace('<0.1',0.0000001) # Ne fonctionne pas !!!!!! 




# SecuAlim_selected.replace('<0.1',0.000001)

# SecuAlim_selected.loc[SecuAlim_selected.Valeur =='<0.1','Valeur']

# SecuAlim_selected[SecuAlim_selected.loc[SecuAlim_selected.Valeur =='<0.1','Valeur']]=0.000001
# tableZ[tableZ.Zone.str.contains("Chine|Fran") == True]
# SecuAlim_selected[SecuAlim_selected.Valeur.str.contains('<0.1')==True]
# SecuAlim_selected.Valeur.astype(float)
# SecuAlim_selected.describe(include='all')
# SecuAlim_selected.info()

In [ ]:
# SecuAlim_selected[SecuAlim_selected.loc[SecuAlim_selected.Valeur =='<0.1','Valeur']]=0.000001   # Pas ok 

# SecuAlim_selected.loc[SecuAlim_selected['Valeur']=='<0.1','Valeur']=0.000001  # A value is trying to be set on a copy of a slice from a DataFrame.
# SecuAlim_selected['Valeur']=='<0.1']='0.000001' 

# SecuAlim_selected.replace({'<0.1': '0.000001'}) # PAS OK un nouvelle colonne 




# SecuAlim_selected['Valeur']=='<0.1'
# SecuAlim_selected[SecuAlim_selected.Valeur =='<0.1'] #OK

In [ ]:
SecuAlim_selected[(SecuAlim_selected.Valeur =='<0.1') & (SecuAlim_selected.AnnéeREF==2000)]
SecuAlim_selected[(SecuAlim_selected.AnnéeREF==2000)].head(40)

In [ ]:
SecuAlimP=SecuAlim_selected.pivot( index=['CodeZone','Zone','AnnéeREF'],columns=['ListeProduit'], values='Valeur')  

SecuAlimP

In [ ]:
# DEBUG  recherche NaN   - OK 
temp=SecuAlimP
temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
len(test)
# test.Zone.unique()

In [ ]:
SecuAlim_selected.describe(include='all')

In [ ]:
# DEBUG  recherche NaN  
temp=SecuAlim_selected
temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

In [ ]:
#FAO  Securité Alimentaire   https://www.fao.org/faostat/fr/#data/FS 
# Rubrique Toutes les données 	https://fenixservices.fao.org/faostat/static/bulkdownloads/Données_de_la_sécurité_alimentaire_F_Toutes_les_Données_(Normalisé).zip
# Fichier unzip & converti en UTF8 
SecuAlimRAW=pd.read_csv(DataPATH+'SecuAlimUTF8.csv',sep=';')

# Formatage SecuAlimRAW
SecuAlimRAW.rename(columns={'Code zone' : 'CodeZone', 'Code Produit': 'CodeProduit', 'Code Élément':'CodeElement','Élément':'Element', 'Code année':'AnnéeREF'},inplace=True)
SecuAlimRAW.drop(columns=['Note','Element'],inplace=True)
#DEBUG SecuAlimRAW


# Converions Période (Année) en AnnéeREF ( TEST pour isoler les Periodes  &  moyenne des deux bornes pour déterminer l'AnnéeREF)
SecuAlimRAW.loc[SecuAlimRAW.AnnéeREF >=3000,'AnnéeREF']=((SecuAlimRAW.loc[SecuAlimRAW.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[:4].astype(int)+SecuAlimRAW.loc[SecuAlimRAW.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[4:8].astype(int))/2).astype(int)

## DEBUG    Requettes  partielles 
#            SecuAlimRAW.AnnéeREF.unique()  
#                   SecuAlimRAW.loc[SecuAlimRAW.AnnéeREF >=3000,'AnnéeREF'].unique()
#                   SecuAlimRAW.loc[SecuAlimRAW.AnnéeREF >=3000,'AnnéeREF'].astype(str).str[:4].astype(int)
#SecuAlimRAW.AnnéeREF.unique() #test pour valider Code 

SecuAlimRAW.head(5)

In [ ]:
# DEBUG 
print(SecuAlim.Zone.unique(),'\nTotal:',len(SecuAlim.Zone.unique()))

In [ ]:
# DEBUG  recherche NaN 
temp=SecuAlimRAW
temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
test.Produit.unique()

In [ ]:
# DEBUG SecuAlimRAW.info();SecuAlimRAW.columns

In [ ]:
# DEBUG print(SecuAlimRAW.Zone.unique(),'\nTotal:',len(SecuAlimRAW.Zone.unique()))

##### 2 Nettoyage

In [ ]:
#Nettoyage  Restriction SecuAlimRAW
# Drop  les CodeZone correspondant à des agrégats ( continent, groupe de pays ........... ) =>  CodeZone>=400 ! 
''' 'Pays les moins développés' 'Pays sans littoral en voie de développement' 'Petites îles en voie de Développement'  'Pays à faible revenu et à déficit vivrier' 'Pays à faible revenu'  'Pays à revenu moyen' 'High-income economies'  'Upper-middle-income economies' '''
# DEBUG  print(SecuAlimRAW.Zone.unique(),'\nTotal:',len(SecuAlimRAW.Zone.unique()

df=SecuAlimRAW
df.drop(df.loc[df['CodeZone'] >= 400].index, inplace=True)
#DEBUG print(df.Zone.unique(),'\nTotal:',len(df.Zone.unique())) #   Verfier Si le filtre a bien fonctionné


df['ListeProduit']=df.CodeProduit.astype(str)+'-'+df.Produit # Creation Colonne listeProduit pour filtrage 
df.ListeProduit.unique()


In [ ]:
# Restriction avec un Masque pour choisir les Codeproduits à afficher
#df_mask=(df.CodeProduit==21010)

df_mask=(df.CodeProduit==21013)|(df.CodeProduit==21014)|(df.CodeProduit==210011)|(df.CodeProduit==21056)|(df.CodeProduit==21057)|(df.CodeProduit==21061)
# df_mask=(df.CodeProduit==21013)|(df.CodeProduit==21014)|(df.CodeProduit==210011)|(df.CodeProduit==210071)|(df.CodeProduit==210081)|(df.CodeProduit==21056)|(df.CodeProduit==21057)|(df.CodeProduit==21061) #Ancienne liste

filtered_df = df[df_mask]

filtered_df.query('AnnéeREF==2010').head(20)
#DEBUG print(filtered_df.CodeProduit.unique(),'\n',filtered_df.ListeProduit.unique())  # Vérif CodeProduit sélectionnées ! 



In [ ]:
# DEBUG  Recheches des Années dispo  pour chaque CodeProduit (Non dispo = futur NaN)

listeCodeProduit=filtered_df.CodeProduit.unique()
for p in listeCodeProduit:
  print(p,' : ',filtered_df[(filtered_df.CodeProduit == p) & (filtered_df.AnnéeREF >= 2000)].AnnéeREF.unique(),'\n' )


# DEBUG Requettes Intermediaires pour faire la boucle For
# filtered_df[(filtered_df.CodeProduit == 210081) & (filtered_df.AnnéeREF >= 2000)].head(20)
# filtered_df[(filtered_df.CodeProduit == 210081) & (filtered_df.AnnéeREF >= 2000)].AnnéeREF.unique()



In [ ]:
df=filtered_df
#Note df.columns  # Index(['CodeZone', 'Zone', 'CodeProduit', 'Produit', 'CodeElement', 'AnnéeREF', 'Année', 'Unité', 'Valeur', 'Symbole', 'ListeProduit'],     dtype='object')

# Creation du Pivot table  pour basculer les lignes en colonnes  et DF multi-index pour visualiser 
tableP=df.pivot( index=['CodeZone','Zone','AnnéeREF'],columns=['ListeProduit'], values='Valeur')    # tableP=df.pivot( index=['CodeZone','Zone','AnnéeREF', 'Année'],columns=['ListeProduit'], values='Valeur')
tableP
#tableZ=df.pivot( index=['CodeZone','Zone','Année','AnnéeREF'],columns=['ListeProduit'], values='Valeur')


In [ ]:
# Convertion df multi-index  vers  df index simple
tableZ=tableP

tableZ.columns=tableZ.columns.to_flat_index() # colonnes => concatener le multi index en un index simple https://datascientyst.com/flatten-multiindex-in-pandas/
#tableZ.columns = tableZ.columns.get_level_values(0) + '_' +  tableZ.columns.get_level_values(1).astype(str)+ '_' + tableZ.columns.get_level_values(2)

tableZ=tableZ.reset_index() # lignes Conversion de multi-index en colonnes.

#Iterration de toutes  colonnes  du dataFrame pour remplacer une valeur 
for (colname,colval) in tableZ.iteritems():
    # print(colname, colval.values)
    tableZ.loc[tableZ[colname] =='<0.1', colname] = 0.000001 

tableZ.CodeZone=tableZ.CodeZone.astype(int)
tableZ.AnnéeREF=tableZ.AnnéeREF.astype(int)
#tableZ[['21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)','21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)']].astype(float)
pd.to_numeric(tableZ['21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)'])
# tableZ

# Jointure pour data visualisation sur map 
# GeoTable=tableZ.merge(PAYS, left_on='CodeZone',right_on='Code_Pays')
# GeoTable.drop(['CodeContinent','Code_ZoneContinent','Code_Pays','Pays'], axis=1)

tableZ=tableZ.merge(PAYS, left_on='CodeZone',right_on='Code_Pays').drop(['CodeContinent','Code_ZoneContinent','Code_Pays','Pays'], axis=1) #.fillna(0)



tableZ.CodeZone=tableZ.CodeZone.astype(int)
tableZ.AnnéeREF=tableZ.AnnéeREF.astype(int)
tableZ[['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)',
       '21013-Disponibilités protéiques moyennes (g/personne/jour) (moyenne sur 3 ans)',
       '21014-Disponibilités protéines moyennes d’origine animale (g/personne/jour) (moyenne sur 3 ans)',
       '21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)',
       '21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)',
       '21061-Disponibilités moyennes de matière grasse (g/personne/jour) (moyenne sur 3 ans)']]=tableZ[['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)',
       '21013-Disponibilités protéiques moyennes (g/personne/jour) (moyenne sur 3 ans)',
       '21014-Disponibilités protéines moyennes d’origine animale (g/personne/jour) (moyenne sur 3 ans)',
       '21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)',
       '21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)',
       '21061-Disponibilités moyennes de matière grasse (g/personne/jour) (moyenne sur 3 ans)']].astype(float)
# tableZ['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)']=tableZ['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)'].astype(float)
tableZ=tableZ.fillna (0)

tableZ.info()

#### test des NaN  

In [ ]:
# DEBUG  recherche NaN   - OK 
temp=tableZ
temp.isnull().sum(axis=0) # Total nombre de NaN par colonne
# test=temp[temp.isnull().any(axis=1)] ; test #Afficher les lignes ayant au moins 1 NaN
# len(test)
# test.Zone.unique()

#### test des NaN  ancienne Version 

In [ ]:
tableZ[tableZ.isnull().any(axis=1)]

In [ ]:
tableZ.isna().any() #tableZ.isnull().any()

In [ ]:
tableZ.isnull().any(axis=1) # equivalent : tableZ.isna().any(axis=1)

In [ ]:
tableZ.isnull().sum(axis=0)

In [ ]:
tableZ.info()

In [ ]:
#Iteration data frame  (toutes les valeurs &  Sans Doublons)
for (colname,colval) in tableZ.iteritems():
    # print(colname, colval.values)
    print(colname, colval.unique())

#Graphes

### Test Graph1

In [ ]:
#INFO tableZ.columns
""" Index(['CodeZone', 'Zone', 'AnnéeREF',
       '210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)',
       '21013-Disponibilités protéiques moyennes (g/personne/jour) (moyenne sur 3 ans)',
       '21014-Disponibilités protéines moyennes d’origine animale (g/personne/jour) (moyenne sur 3 ans)',
       '21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)',
       '21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)',
       '21061-Disponibilités moyennes de matière grasse (g/personne/jour) (moyenne sur 3 ans)'],
      dtype='object', name='ListeProduit')
"""

import plotly.express as px
fig = px.line(tableZ, x='AnnéeREF', y='210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)', color='Zone', markers=True)
fig.show()

### Seaborn Violin demo 

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid")

# Load the example tips dataset
tips = sns.load_dataset("tips")

# Draw a nested violinplot and split the violins for easier comparison
sns.violinplot(data=tips, x="day", y="total_bill", hue="smoker",
               split=True, inner="quart", linewidth=1,
               palette={"Yes": "b", "No": ".85"})
sns.despine(left=True)

In [ ]:
tips

In [ ]:
tableZ.describe()
# tableZ.describe(include='all')

### Bilan Alimentaire

#### Formatage et Netttoyage des données

In [ ]:
BilanAlimRAW.info()

In [ ]:
BilanAlimRAW

# Plotly Demo 

### Plotly demo map with animation

In [ ]:
g = sns.PairGrid(tableZ)
g.map_diag(sns.histplot)

In [ ]:
tableZ.plot.hist(bins=100)

In [ ]:
tableZ.plot()

In [ ]:
tableZ.hist(bins=100,color="green") ; plt.show()

In [ ]:
tableZ['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)'].plot.hist()


In [ ]:
tableZ['21013-Disponibilités protéiques moyennes (g/personne/jour) (moyenne sur 3 ans)'].plot.hist()

In [ ]:
tableZ.columns

In [ ]:
#Iteration data frame  (toutes les valeurs &  Sans Doublons)
for (colname,colval) in tableZ.iteritems():
    # print(colname, colval.values)
    print(colname, colval.unique())


In [ ]:
tableZ.columns

In [ ]:
import plotly.express as px

fig = px.scatter_geo(tableZ, locations="AlphaISO3", color="ZoneContinent",
                     hover_name="Zone", size="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",
                     animation_frame="AnnéeREF",
                     projection="natural earth")
fig.show()

In [ ]:
import plotly.express as px

fig = px.scatter_geo(tableZ, locations="AlphaISO3", color="ZoneContinent",
                     hover_name="Zone", size="21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)",
                     animation_frame="AnnéeREF",
                     projection="natural earth")
fig.show()

In [ ]:
px.strip(tableZ, x="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",hover_name='Zone',color="continent")

In [ ]:
px.histogram(tableZ, x="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",hover_name='Zone',color="continent",marginal="rug")

In [ ]:
px.histogram(tableZ, x="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",hover_name='Zone',color="continent",marginal="rug",facet_col="continent")

In [ ]:
px.histogram(tableZ, x="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",y='AnnéeREF',hover_name='Zone',color="continent",marginal="rug")

In [ ]:
tableZ.columns

In [ ]:
px.bar(tableZ, color="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",x="AnnéeREF",y="continent",hover_name="Zone")

In [ ]:
px.sunburst(tableZ, color="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",values="AnnéeREF",path=["continent","Zone"],hover_name="Zone", height=800)

In [ ]:
px.treemap(tableZ, color="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)",values="AnnéeREF",path=["continent","Zone"],hover_name="Zone", height=800)

In [ ]:
px.choropleth(tableZ, color="21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)", locations="AlphaISO3",hover_name="Zone", height=500)

In [ ]:
fig = px.scatter_geo(tableZ.query("AnnéeREF==2017"), locations="AlphaISO3", color="continent",
                     hover_name="Zone", size="21057-Besoins énergétiques alimentaires moyens (kcal/personne/jour)", projection="natural earth")
fig.show()

In [ ]:
import plotly.express as px

tableG=tableZ
tableG['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)']
# tableG['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)']=tableG[('210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)'>=0.0)]
# fig = px.scatter_geo(tableZ, locations="AlphaISO3", color="ZoneContinent",
#                      hover_name="Zone", size="210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)",
#                      animation_frame="AnnéeREF",
#                      projection="natural earth")
# fig.show()

### Exemple

In [ ]:
import plotly.express as px
df = px.data.gapminder()
fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     animation_frame="year",
                     projection="natural earth")
fig.show()

##### Plotly Demo1 

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("continent == 'Oceania'")
fig = px.line(df, x='year', y='lifeExp', color='country', markers=True)
fig.show()

In [ ]:
df = px.data.gapminder().query("continent == 'Oceania'");df

In [ ]:
import plotly.express as px
df = px.data.stocks(indexed=True)
fig = px.line(df, facet_row="company", facet_row_spacing=0.01, height=200, width=200)

# hide and lock down axes
fig.update_xaxes(visible=False, fixedrange=True)
fig.update_yaxes(visible=False, fixedrange=True)

# remove facet/subplot labels
fig.update_layout(annotations=[], overwrite=True)

# strip down the rest of the plot
fig.update_layout(
    showlegend=False,
    plot_bgcolor="white",
    margin=dict(t=10,l=10,b=10,r=10)
)

# disable the modebar for such a small plot
fig.show(config=dict(displayModeBar=False))

In [ ]:
df

# DIVers Code

In [ ]:
tableZ[tableZ.Zone.str.contains("Chine|Fran") == True]

In [ ]:
#DEBUG 
DECRIRE(tableZ)

In [ ]:



#tableZ=df.pivot( index=['CodeZone','Zone','Année','AnnéeREF'],columns=['ListeProduit'], values='Valeur')

tableZ=df.pivot( index=['CodeZone','Zone','AnnéeREF'],columns=['ListeProduit'], values='Valeur')

tableZ.columns=tableZ.columns.to_flat_index() # colonnes => concatener le multi index en un index simple https://datascientyst.com/flatten-multiindex-in-pandas/
#tableZ.columns = tableZ.columns.get_level_values(0) + '_' +  tableZ.columns.get_level_values(1).astype(str)+ '_' + tableZ.columns.get_level_values(2)

tableZ=tableZ.reset_index() # lignes Conversion de multi-index en colonnes.

tableZ.fillna(0, inplace=True)# ATTENTION  NaN remplacé par des zéro ( 0 <=> NaN)

#Iterration de toutes  colonnes  du dataFrame pour remplacer une valeur 
for (colname,colval) in tableZ.iteritems():
    #print(colname, colval.values)
    tableZ.loc[tableZ[colname] =='<0.1', colname] = 0.000001 

#tableZ.CodeZone=tableZ.CodeZone.astype(int)
#tableZ.AnnéeREF=tableZ.AnnéeREF.astype(int)
tableZ.rename(index={'ListeProduit':'index'}, inplace=True)


tableZ2=tableZ

# Changment manuel dans une colonne 
#tableZ.loc[tableZ['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)'] =='<0.1', '210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)'] = 0.000001 ; #snut.snut_millionsHab=snut.snut_millionsHab.astype(float)
#tableZ['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)']=tableZ['210011-Nombre de personnes sous-alimentées (millions) (moyenne sur 3 ans)'].astype(float)

In [ ]:


# DEBUG tableZ[tableZ['Zone'].str.contains('Fr') & (tableZ.AnnéeREF>=2017)]

print(tableZ['21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)'].isnull().sum(),
      tableZ['21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)'].notnull().sum(),
    
 
)
#type(tableZ['21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)'])


#tableZ['21056-Besoins énergétiques alimentaires minimaux (kcal/personne/jour)'].fillna(0).describe()



tableZ.describe(include='all')
tableZ2.info()

In [ ]:
# DEBUG tableZ.describe(include='all')  

In [ ]:
tableZ.info()

In [ ]:
df.info()

In [ ]:
# Reshape du dataframe
#table = pd.pivot_table(SecuAlimRAW,index=['CodeZone','Zone','AnnéeREF'], columns=['CodeProduit','Produit'])
table = pd.pivot_table(SecuAlimRAW,index=['CodeZone','Zone','AnnéeREF',	'CodeAnnée','Année'], columns=['CodeProduit','Produit']) ;table

In [ ]:
# Converstion du df multi index en dataframe index simple 
tableZ=table
 
# tableZ.columns=tableZ.columns.to_flat_index() # colonnes => concatener le multi index en un index simple https://datascientyst.com/flatten-multiindex-in-pandas/
tableZ.columns = tableZ.columns.get_level_values(0) + '_' +  tableZ.columns.get_level_values(1).astype(str)+ '_' + tableZ.columns.get_level_values(2)

tableZ=tableZ.reset_index() # lignes Conversion de multi-index en colonnes.
tableZ

#https://techblog.deepki.com/dataframe-multi-index/

In [ ]:
tableZ["CodeElement_21048_Pourcentage de la population utilisant au moins les services d'assainissement de base"].unique()

In [ ]:
tableX=tableZ.mask(tableZ.CodeZone>=5000) 
tableX.dropna();tableX


## DataSet [OC]

In [ ]:
#  Importer les données en Pandas dataframe
#aideDataPATH='/content/drive/MyDrive/DataSc/P4VA/'
DataPATH='https://raw.githubusercontent.com/vincent-arese/OC-P04VA/main/data/'

aide=pd.read_csv(DataPATH+'aide_alimentaire.csv')  #OK
dispo=pd.read_csv(DataPATH+'dispo_alimentaire.csv')
population=pd.read_csv(DataPATH+'population.csv') #ok
snut=pd.read_csv(DataPATH+'sous_nutrition.csv')      # en Cours ........
geocode=pd.read_csv(DataPATH+'CodePaysMonde.csv', sep = ';')

In [ ]:
dispo

# Nouvelle section

In [ ]:
snut

# Nouvelle section

https://fenixservices.fao.org/faostat/static/bulkdownloads/BilansAlimentaires_F_Toutes_les_Données_(Normalisé
).zip


https://www.fao.org/faostat/fr/#data/FBS





In [ ]:
#df = pd.read_csv('https://fenixservices.fao.org/faostat/static/bulkdownloads/FoodBalanceSheets_E_All_Data_(Normalized).zip')